# This notebook provides an interactive analsyis to map snow frequency at a site
26 April 2022  
NSh  

In [ ]:
# Import libraries and authenticate with Google Earth Engine.
import os
import ee
import geemap

May need to update the geemap package (only once. Then comment out this line.)

In [ ]:
geemap.update_package()

### Create an interactive map and define region of interest (roi).

In [ ]:
# Define cloud mask function per Earth Engine Catalog.
def mask_S2_clouds(image):
    qa = image.select('QA60')
    
    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloud_bit_mask = 1<<10
    cirrus_bit_mask = 1<<11
    
    # Both flags should be set to zero, indicating clear conditions.
    mask = (qa.bitwiseAnd(cloud_bit_mask)
            .eq(0)
            .And(qa.bitwiseAnd(cirrus_bit_mask)
            .eq(0)))
    
    return image.updateMask(mask).divide(10000)

In [ ]:
# Import dataset and pre-filter to get less cloudy granules.
dataset = (ee.ImageCollection('COPERNICUS/S2_SR')
           .filterDate('2017-03-28', '2022-04-26')
           .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',60))
           .map(mask_S2_clouds))

In [ ]:
# Define visualization parameters for optical imagery
vis_params = {
    'min': 0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

In [ ]:
# Create interactive map centered at region of interest (roi)
Map = geemap.Map()
Map.addLayer(dataset.mean(), vis_params, 'RGB')
roi = ee.Geometry.Point([7.277208, 61.965656]) #Ryggfonn
Map.centerObject(roi, 10)
Map